# Exercício sobre otimização hiperparamétrica para regressão de preços de residências usando redes neurais densas (DNNs)

Neste exercício, você irá encontrar o melhor conjunto de hiperparâmetros de um modelo para **regressão**.

Leia atentamente o conteúdo de todas as células e sigas as intruções abaixo.

Ao final, responda:

+ Qual o melhor conjunto de hiperparâmetros encontrado?
+ Esse resultado é melhor do que aquele que encontramos em sala de aula?

(**Justifique todas as suas respostas**).

Respostas:

1) O otimizador ideal encontrado é o adam.

2) O resultado encontrado neste exercício é melhor, pois ele obteve um menor erro de validação, que foi o criterio escolhido para comparação. Portanto, ele generaliza melhor do que o que encontramos em sala de aula.

## Importe as bibliotecas

Execute a célula abaixo.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# Importamos a classe StandardScaler.
from sklearn.preprocessing import StandardScaler

# Instalando o KerasTuner.
!pip install keras-tuner --upgrade

# Importanda a biblioteca KerasTuner.
import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


## Baixe a base de dados

Execute a célula de código abaixo.

### A base de dados

A base de dados contém **informações do censo dos EUA relativos aos preços de residências** em diferentes locais nos subúrbios de Boston **no final dos anos 1970**.

**A base possui 506 exemplos e 13 atributos numéricos** (atributos $x_i$ com $i$ variando de 1 a 13).

O **décimo quarto atributo** (i.e., MEDV: Median value of owner-occupied homes in USD 1000's) é considerado como sendo o **rótulo**, ou seja, o valor que queremos predizer com o modelo.

A descrição das informações contidas no banco de dados segue abaixo.

| Attribute |                              Description                              |
|:---------:|:---------------------------------------------------------------------:|
|    CRIM   |                     per capita crime rate by town                     |
|     ZN    |    proportion of residential land zoned for lots over 25,000 sq.ft.   |
|   INDUS   |            proportion of non-retail business acres per town           |
|    CHAS   | Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) |
|    NOX    |           nitric oxides concentration (parts per 10 million)          |
|     RM    |                  average number of rooms per dwelling                 |
|    AGE    |         proportion of owner-occupied units built prior to 1940        |
|    DIS    |          weighted distances to five Boston employment centres         |
|    RAD    |               index of accessibility to radial highways               |
|    TAX    |                full-value property-tax rate per USD 10.000               |
|  PTRATIO  |                      pupil-teacher ratio by town                      |
|     B     |  1000(Bk - 0.63)^2 where Bk is the proportion of black people by town |
|   LSTAT   |                     lower status of the population                    |
|    MEDV   |            Median value of owner-occupied homes in USD 1000's            |


### Objetivo

O objetivo é encontrar um modelo de **regressão** que prediga o valor médio das casas naquela área usando os 13 atributos fornecidos.

In [2]:
data = tf.keras.datasets.boston_housing

(x_train, y_train), (x_test, y_test) = data.load_data()

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Pradronize os dados

Execute a célula abaixo.

In [3]:
# Instanciamos um objeto da classe StandardScaler.
scaler = StandardScaler()

# Calcula-se os parâmetros de padronização usando o conjunto de treinamento.
scaler.fit(x_train)

# Usamos o método `transform` para padronizar os atributos de treinamento e teste.
x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

## Encontre os melhores hiperparâmetros

### Defina a função de criação do modelo


Modifique a função abaixo de forma que a busca aleatória teste combinações aleatórias dos seguintes hiperparâmetros e valores:

1.   Número de camadas ocultas (`layers`): 1, 2 e 3.
2.   Número de neurônios nas camadas ocultas (`units`): 5, 10, 15, 20 e 25.
3.   Passo de aprendizagem do otmizador (`learning_rate`): 0.0003, 0.001 e 0.003.
4.   Otimizador (`optimizer`): 'sgd' e 'adam'.
5.   Função de ativação das camadas ocultas (`activation`): 'relu' e 'sigmoid'.

**Observações**

+ O keras tuner oferece várias classe para a variação dos hiperparâmetros, as quais podem ser acessadas em: https://keras.io/api/keras_tuner/hyperparameters/
+ O parâmetro `name` de cada hiperparâmetro deve ser único conforme consta na documentação do keras tuner: `name: Must be unique for each HyperParameter instance in the search space.`. Acesse o link acima para ler a documentação.
+ A busca deve demorar vários minutos, tenha paciência.

In [7]:
def build_model(hp):

  # Cria o modelo sequêncial.
  model = tf.keras.Sequential()

  # Adicionando a camada de entrada. Ela não possui pesos, apenas especifica as dimensões da entrada.
  model.add(tf.keras.layers.Input(shape=(13,)))


  # ADICIONE O SEU CÓDIGO AQUI.

  # Hiperparâmetros variáveis:
  num_layers = hp.Int('num_layers', min_value=1, max_value=3)
  activation = hp.Choice('activation', values = ['relu', 'sigmoid'])

 # Adiciona camadas ocultas com unidades e ativação definidos
  for i in range(num_layers):
    units = hp.Choice(f'unit_layer_{i}', values = [5,10,15,20,25])
    model.add(tf.keras.layers.Dense(units=units,activation=activation))

  # Camada de saída
  model.add(tf.keras.layers.Dense(1))

  # Otimizador
  optimizer_choice = hp.Choice('optimizer', values = ['adam', 'sgd'])
  leaerning_rate = hp.Choice('learning_rate', values = [0.0003, 0.001, 0.003])

  if optimizer_choice == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=leaerning_rate)
  else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=leaerning_rate)


  # Compila o modelo.
  model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

  # Retorna o modelo.
  return model

### Instancie o objeto de busca aleatória

Execute as células abaixo.

In [8]:
# Instanciando um objeto da classe RandomSearch.
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss'
)

#### Faça a busca pelo melhor modelo.

Execute as células abaixo.

In [9]:
tuner.search(
    x_train_std, y_train,
    epochs=500,
    validation_data=(x_test_std, y_test)
)

Trial 10 Complete [00h 01m 42s]
val_loss: 22.734281539916992

Best val_loss So Far: 14.563196182250977
Total elapsed time: 00h 17m 36s


Obtendo os melhores hiperparâmetros.

**Observação**
+ Você deverá alterar o código abaixo dependendo de como você implementou a variação dos hiperparâmetros.

In [12]:
best_hps  = tuner.get_best_hyperparameters(1)[0]

print('A pesquisa de hiperparâmetros foi concluída.')
print(f"""O número ideal de camadas ocultas é {best_hps.get('num_layers')}.""")
for i in range(best_hps.get('num_layers')):
    print(f"""O número ideal de neurônios na camada oculta # {i+1} é {best_hps.get('unit_layer_'+str(i))}.""")
print(f"""O valor ideal do passo de aprendizagem é o {best_hps.get('learning_rate')}.""")
print(f"""O otimizador ideal é {best_hps.get('optimizer')}.""")
print(f"""A função de ativação ideal é a {best_hps.get('activation')}.""")

A pesquisa de hiperparâmetros foi concluída.
O número ideal de camadas ocultas é 2.
O número ideal de neurônios na camada oculta # 1 é 25.
O número ideal de neurônios na camada oculta # 2 é 5.
O valor ideal do passo de aprendizagem é o 0.003.
O otimizador ideal é adam.
A função de ativação ideal é a relu.


Imprimindo o resumo dos resultados da busca.

In [13]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 03 summary
Hyperparameters:
num_layers: 2
activation: relu
unit_layer_0: 25
optimizer: adam
learning_rate: 0.003
unit_layer_1: 5
Score: 14.563196182250977

Trial 06 summary
Hyperparameters:
num_layers: 1
activation: relu
unit_layer_0: 20
optimizer: adam
learning_rate: 0.001
unit_layer_1: 20
unit_layer_2: 20
Score: 17.77898406982422

Trial 08 summary
Hyperparameters:
num_layers: 2
activation: relu
unit_layer_0: 15
optimizer: adam
learning_rate: 0.0003
unit_layer_1: 15
unit_layer_2: 20
Score: 21.053327560424805

Trial 01 summary
Hyperparameters:
num_layers: 2
activation: sigmoid
unit_layer_0: 15
optimizer: sgd
learning_rate: 0.001
unit_layer_1: 10
Score: 22.623092651367188

Trial 09 summary
Hyperparameters:
num_layers: 2
activation: sigmoid
unit_layer_0: 20
optimizer: sgd
learning_rate: 0.0003
unit_layer_1: 25
unit_layer_2: 20
Score: 22.734281539916992

Trial 05 summary

## Obtenha e construa o melhor modelo

Execute a célula abaixo.

In [15]:
best_model = tuner.get_best_models(1)[0]

best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 25)             │           350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486 (1.90 KB)

 Trainable params: 486 (1.90 KB)

 Non-trainable params: 0 (0.00 B)

## Avalie o melhor modelo nos conjuntos de treinamento e teste.

Execute as células abaixo.

In [16]:
train_eval = best_model.evaluate(x_train_std, y_train)

print ("MSE no conjunto de treinamento: {:.5}".format(train_eval[0]))
print ("MAE no conjunto de treinamento: {:.5}".format(train_eval[1]))

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.5641 - mae: 1.4229  
MSE no conjunto de treinamento: 4.3753
MAE no conjunto de treinamento: 1.5215


In [17]:
test_eval = best_model.evaluate(x_test_std, y_test)

print ("MSE no conjunto de teste: {:.5}".format(test_eval[0]))
print ("MAE no conjunto de teste: {:.5}".format(test_eval[1]))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.5638 - mae: 2.2887
MSE no conjunto de teste: 14.563
MAE no conjunto de teste: 2.5355
